# Goal<br>
* Develop a machine learning model using bag of words and vectorization that will take in Netflix descriptions of shows in it's database and predict wether the show matching each discription belongs to a given genre<br>

* Inestigate wether this technique can be inhanced by identifying "unique words," words that only occur in discriptions belonging to that genre

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer


import sklearn.preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression



import warnings
warnings.filterwarnings("ignore")

import wrangle as w
import explore as e
import model as m

#pd.set_option('display.max_colwidth', None)

# Wrangle

Data acquired from kaggle 
Each represints a show featured by Netflix

Columns not representing show descriptions or genres were dropped
A boolean column for each genre appearing in the "genres" column was added signifying if each show belonged to that genre

Rows containning blank descriptions were dropped
'Western' column was dropped do to low represintation (only 41 rows)

Data contains 5791 rows after being prepared 

Note:
    
* adding words two letters and fewer to those removed by stopwords, on next iteration investigate for useful words
* using raw counts for frequency measure, on next iteration consider using percentage

In [2]:
#acquire and prepare data
df = w.get_show_data()

# split data into train, validate and prep
train, validate, test = w.split_my_data(df)

# get set of non-western genres
gen_set = e.get_gens(train)

# print number ow rows in train
print(f"Train set contains {train.shape[0]} rows of data")
print(f"Validate set contains {validate.shape[0]} rows of data")
print(f"Test set contains {test.shape[0]} rows of data")

Train set contains 3242 rows of data
Validate set contains 1390 rows of data
Test set contains 1159 rows of data


In [3]:
train.head()

,description,genres,crime,documentation,history,action,horror,scifi,music,reality,european,thriller,comedy,fantasy,war,romance,animation,family,sport,drama
0,crippling payday loan car cheat emission test ...,"[documentation, crime]",True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,filmmaker kip andersen uncovers secret prevent...,[documentation],False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,azerbaijan layla indonesian scholar fall samir...,"[romance, drama]",False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True
3,quebec impulsive headstrong nelly maloye novic...,"[animation, comedy]",False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False
4,hit hardwood east los angeles coach strong con...,"[documentation, sport]",False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False


KeyError: "['description_lem'] not in index"

In [ ]:
# getting exploration variables

# gets dictionaries 
# containing word, relative requency of apearance in comedy and non-comedy films by count and by document count 
# and a list for each containing frequency numbers only
freq_doc, freq_count, list_freq_doc, list_freq_count = e.get_majoriety_counts(train)

# Explore

**1) Investigate data and choose a genre as a test case**

   * Test case should have a large represintation in the data
   * Test case should have a large number of unique words
   * Test case should have a large number of unique words per document

**How many shows are there of each genre?**

In [ ]:
# get distribution of genres
e.shows_per_gen(train, gen_set)

Comedy and Drama seem to be the front runners.

**How many words in each genre are unique words? (Appear only in descriptions of movies of that fall under that genre)**

In [ ]:
# get number of unique words fro each genre
e.unique_words_per_gen(train, gen_set)

Comedy and Drama seem to be the front runners for unique words as well.

**How fequently do unique words appear in each genre?**

In [ ]:
# get frequency of unique words pre genre
e.unique_words_frequency(train, gen_set)

Not a great deal of variance in unique word frequency. Comedy and Drama both have grater than three they were front runners in the other two criteria

**Comedy** and **Drama** have consistantly high values in all test criteria.
Comedy seems the less vague of the two genres, by human understanding. I will choose **Comedy** as my test case.

**2) Investigate test case (Comedy) and see how it compares to the rest of the data**

* What percent of the data is classified as a Comedy?
* Are there any words that appear more or less frequently across comedy or non-comedy show descriptions? <br>(Overall/by document)

**How much of the data is Comedy?**

In [ ]:
e.omni_pie(train.comedy, "40% of the Training Data are Comedy Titles")

**Can we use the relative frequency of words in descriptions to seperate noise from signal?**

**The following chart shows a histogram of the relative document frequency of each unique word appearing in film descriptions**
<br>
The number is calculated in the following way: <br>
number of comedy documents the word appears in minus the number of non-comedy documents the word appears in

In [ ]:
e.get_hist_doc(list_freq_doc)

* Data appears to normalize around 0 however the steps are steeper on the positive side
* Skew is likely do to the 60/40 imbalance in the data

**The following chart shows the relative frequency of each unique word appearing film descriptions**
The number is calculated in the following way: <br>
number of times the word appears in comedy documents minus number of times the word appears in non-comedy documents

In [ ]:
e.get_hist_word(list_freq_count)

* Data appears to normalize around -2 and there seems to be more negative values than positive ones
* Skew is likely do to the 60/40 imbalance in the data

Words that do not have a significant difference in apearances between comedy and non-comedy films will not be useful in guiding the model. Looking at the results of these distributions tells me that once the data is vectorized there will be a lot of noise words that can be dropped to make sure that signal words are having a stronger impact on the model.

**As a quick sanity test lets take a look at our top 15 most extream values for document and word frequency**

In [ ]:
e.get_doc_ext(dict(freq_doc[:15]), "Top 15 Most Extreme Negative Document Frequency Values")

In [ ]:
e.get_doc_ext(dict(freq_count[:15]), "Top 15 Most Extreme Negative Word Frequency Values")

In [ ]:
e.get_doc_ext(dict(freq_doc[-15:]), "Top 15 Most Extreme Positive Document Frequency Values")

In [ ]:
e.get_doc_ext(dict(freq_count[-15:]), "Top 15 Most Extreme Positive Document Frequency Values")

* Extreme negative values seem suspect due to the imbalance in the data words commen to both comedy and non-comedy films may have a large negative value because there are more non-comedy films in the data.
* Extreme positive value words are intuatively indicitive of the comedy genre and servive in spite of the data imbalance working against them. They seem like good indicators.

# Modeling

I will be examining different tequniques for building the most accurate possible model. Techniques include using word count vs TF-IDF, differint machine learning classification models, and a number of feature engineering ideas. For this reason models will be graded on overall accuracy above all. Ties will be decided by fewist false negatives. As a buisness case, it seems better to me to present a movie to a customer out of genre then to fail to present an in ingenre film that might increase that customers watchtime, provided these occurances are kept to a minimum.
Because the data has a 60/40 split my beginning baseline is 60% and will hopefully be improved on throughout the modeling process.

In [ ]:
# seperate train, validate and test data into X (description) and y (comedy) 
train_X = train[['description']].reset_index(drop=True)
train_y = train[['comedy']].reset_index(drop=True)

validate_X = validate[['description']].reset_index(drop=True)
validate_y = validate[['comedy']].reset_index(drop=True)

test_X = test[['description']].reset_index(drop=True)
test_y = test[['comedy']].reset_index(drop=True)

## Count Analysis - Full Vectorization Array

In [ ]:
train_counts, validate_counts, test_counts = m.get_counts(train_X, validate_X, test_X)

In [ ]:
#m.get_acc_table(train_counts, train_y, validate_counts, validate_y)

When predicting on raw counts the logistic regression and random forest models both broke 70% accuracy. Lets see if I can improve on that by cutting out some of the noise. For the next two attempts I will drop all of the words that with a 0 relative frequency for document and raw count frequency. 

## Count Analysis - Drop 0 Relative Frequency - Raw Counts

In [ ]:
train_counts.columns.to_list()

In [ ]:
train_counts['wilson']

In [ ]:
drop_lst = []

for freq in freq_count:
    
    if freq[1] == 0:
        
        drop_lst.append(freq[0])


print(len(train_counts), len(drop_lst))

for 
train_counts.drop(columns=drop_lst, inplace = True)
validate_counts.drop(columns=drop_lst)

print(len(train_counts))
